# Supplementary Notebook S05: Stoichiometric Model of WL-Pathway with Sinks

This notebook is supplementary to the the research paper titled:

Optimizing acetogenic CO(2) utilization through metabolic and process engineering highlights the potential of thermophilic bioproduction

In this notebook, a stoichiometric model of the Wood-Ljungdahl pathway is created, using an existing Genome-Scale Metabolic Model of Clostridium ljungdahlii. Bifurcating enzymes restoring the energy and redox co-factors are not included, the co-factors are created or removed from the model using sink reactions.

In [1]:
import cobra.io
import escher
from escher import Builder
from cobra import Model, Reaction, Metabolite

In [2]:
Cljung_Model = cobra.io.read_sbml_model('../../Models/PublishedModels/C-ljungdahlii_iHN637.xml')

Using license file /Users/phivri/gurobi.lic
Academic license - for non-commercial use only - expires 2022-07-23


To simplify the model, 3 changes are made:

The ATP maintenance reaction is unconstrained, so that a flux is not forced for non-growth associated maintenance

Acetate transport is dependent on a symporter, since only the core metabolism is investigated here, the reaction is replaced by a more simple transporter.

The reaction MTHFD2 uses NADPH as a cofactor, to simplify the model, this cofactor is replaced by NADH. This way there is only one

In [3]:
Cljung_Model.reactions.ATPM.bounds = 0, 1000

acT = Reaction("ACt")
acT.name = "Acetate transporter"
acT.add_metabolites({ 
    Cljung_Model.metabolites.ac_c: -1.0,
    Cljung_Model.metabolites.ac_e: 1.0
})

Unconstrain reactions that have been diasbled in the C.ljungdahlii model

In [4]:
Cljung_Model.reactions.ACACT1r.bounds = 0,1000.0
Cljung_Model.reactions.HACD1.bounds = 0,1000.0
Cljung_Model.reactions.ECOAH1.bounds = 0,1000.0
Cljung_Model.reactions.ACOAD1z.bounds = 0,1000.0
Cljung_Model.reactions.PBUTT.bounds = 0,1000.0
Cljung_Model.reactions.BUTKr.bounds = -1000.0,0
Cljung_Model.reactions.BTCOARx.bounds = 0,1000.0
Cljung_Model.reactions.ALCD4.bounds = -1000.0,0
Cljung_Model.reactions.BTOHt.bounds = -1000.0, 1000.0
Cljung_Model.reactions.BUTt.bounds = -1000.0, 1000.0

In [5]:
WLmodel = Model('Wood_Ljungdahl_Core')

WLmodel.add_reactions([
#Transport:
Cljung_Model.reactions.H2td,
Cljung_Model.reactions.H2Ot,
Cljung_Model.reactions.CO2t,
Cljung_Model.reactions.COt,
Cljung_Model.reactions.ETOHt,
Cljung_Model.reactions.BTDt_RR,
Cljung_Model.reactions.BUTt,
Cljung_Model.reactions.BTOHt,
    
#Carbon fixation to acetyl-CoA
Cljung_Model.reactions.FDH7,
Cljung_Model.reactions.FTHFLi,
Cljung_Model.reactions.MTHFC,
Cljung_Model.reactions.MTHFD,
Cljung_Model.reactions.MTHFR5,
Cljung_Model.reactions.METR,
Cljung_Model.reactions.CODH4,
Cljung_Model.reactions.CODH_ACS,

#Product formation from acetyl-CoA
Cljung_Model.reactions.PTAr,
Cljung_Model.reactions.ACKr,
Cljung_Model.reactions.ACAFDOR,
Cljung_Model.reactions.ALCD2x,
    
Cljung_Model.reactions.POR,
Cljung_Model.reactions.ACLS,
Cljung_Model.reactions.ACLDC,
Cljung_Model.reactions.BTDD_RR,

Cljung_Model.reactions.ACACT1r,
Cljung_Model.reactions.HACD1,
Cljung_Model.reactions.ECOAH1,
Cljung_Model.reactions.ACOAD1z,
Cljung_Model.reactions.BTCOARx,
Cljung_Model.reactions.ALCD4,
Cljung_Model.reactions.PBUTT,
Cljung_Model.reactions.BUTKr])

#Exchanges and sinks
WLmodel.add_reactions([acT])
WLmodel.add_boundary(WLmodel.metabolites.get_by_id("h2_e"), type="exchange")
WLmodel.add_boundary(WLmodel.metabolites.get_by_id("h2o_e"), type="exchange")
WLmodel.add_boundary(WLmodel.metabolites.get_by_id("co2_e"), type="exchange")
WLmodel.add_boundary(WLmodel.metabolites.get_by_id("co_e"), type="exchange")
WLmodel.add_boundary(WLmodel.metabolites.get_by_id("ac_e"), type="exchange")
WLmodel.add_boundary(WLmodel.metabolites.get_by_id("etoh_e"), type="exchange")
WLmodel.add_boundary(WLmodel.metabolites.get_by_id("btd_RR_e"), type="exchange")
WLmodel.add_boundary(WLmodel.metabolites.get_by_id("but_e"), type="exchange")
WLmodel.add_boundary(WLmodel.metabolites.get_by_id("btoh_e"), type="exchange")

WLmodel.add_boundary(WLmodel.metabolites.get_by_id("atp_c"), type="sink")
WLmodel.add_boundary(WLmodel.metabolites.get_by_id("adp_c"), type="sink")
WLmodel.add_boundary(WLmodel.metabolites.get_by_id("nadh_c"), type="sink")
WLmodel.add_boundary(WLmodel.metabolites.get_by_id("nadp_c"), type="sink")
WLmodel.add_boundary(WLmodel.metabolites.get_by_id("nadph_c"), type="sink")
WLmodel.add_boundary(WLmodel.metabolites.get_by_id("nad_c"), type="sink")
WLmodel.add_boundary(WLmodel.metabolites.get_by_id("fdxo_42_c"), type="sink")
WLmodel.add_boundary(WLmodel.metabolites.get_by_id("fdxr_42_c"), type="sink")
WLmodel.add_boundary(WLmodel.metabolites.get_by_id("coa_c"), type="sink")
WLmodel.add_boundary(WLmodel.metabolites.get_by_id("h_c"), type="sink")
WLmodel.add_boundary(WLmodel.metabolites.get_by_id("pi_c"), type="sink")

WLmodel.reactions

[<Reaction H2td at 0x7fd160b837f0>,
 <Reaction H2Ot at 0x7fd160b71a90>,
 <Reaction CO2t at 0x7fd160a1a160>,
 <Reaction COt at 0x7fd160a55fd0>,
 <Reaction ETOHt at 0x7fd160b02c50>,
 <Reaction BTDt_RR at 0x7fd160856160>,
 <Reaction BUTt at 0x7fd160899780>,
 <Reaction BTOHt at 0x7fd160856cc0>,
 <Reaction FDH7 at 0x7fd160b8b400>,
 <Reaction FTHFLi at 0x7fd160bc6cf8>,
 <Reaction MTHFC at 0x7fd160d31be0>,
 <Reaction MTHFD at 0x7fd160d31f60>,
 <Reaction MTHFR5 at 0x7fd160d3bb70>,
 <Reaction METR at 0x7fd160c293c8>,
 <Reaction CODH4 at 0x7fd160ae1748>,
 <Reaction CODH_ACS at 0x7fd160ae8208>,
 <Reaction PTAr at 0x7fd160ddacc0>,
 <Reaction ACKr at 0x7fd1608f5b38>,
 <Reaction ACAFDOR at 0x7fd160a877b8>,
 <Reaction ALCD2x at 0x7fd140f476a0>,
 <Reaction POR at 0x7fd160ebebe0>,
 <Reaction ACLS at 0x7fd1608e4128>,
 <Reaction ACLDC at 0x7fd160a01198>,
 <Reaction BTDD_RR at 0x7fd160a9d550>,
 <Reaction ACACT1r at 0x7fd160870cf8>,
 <Reaction HACD1 at 0x7fd160b83e80>,
 <Reaction ECOAH1 at 0x7fd160af94e0>,

## Add Acetone production pathway from ABE

In [6]:
WLmodelActn = WLmodel

WLmodelActn.add_metabolites(Metabolite(id='acac_c',formula = 'C4H6O3', name = 'Acetoacetate', compartment = 'c' ))
WLmodelActn.add_metabolites(Metabolite(id='act_c',formula = 'C3H6O', name = 'Acetone', compartment = 'c' ))
WLmodelActn.add_metabolites(Metabolite(id='act_e', formula = 'C3H6O', name = 'Acetone', compartment = 'e'))

In [7]:
ctfAB_ac = Reaction("ctfAB_ac")
ctfAB_ac.name = "Acetate-acetoacetate CoA-transferase"

ctfAB_ac.add_metabolites({
    WLmodelActn.metabolites.ac_c: -1.0,      #Acetate
    WLmodelActn.metabolites.aacoa_c: -1.0,   #Acetoacetyl-CoA
    WLmodelActn.metabolites.acac_c: 1.0,     #Acetoacetate
    WLmodelActn.metabolites.accoa_c: 1.0,    #Acetyl-CoA
    WLmodelActn.metabolites.h_c: -1.0
})
print(ctfAB_ac.check_mass_balance())    #ensure correct mass balance
print(ctfAB_ac.bounds)                  #Ensure correct bounds
print(ctfAB_ac.reaction)                #Ensure correct reaction

{}
(0.0, 1000.0)
aacoa_c + ac_c + h_c --> acac_c + accoa_c


In [8]:
adc = Reaction("ADC")
adc.name = "acetoacetate decarboxylase"

adc.add_metabolites({
    WLmodelActn.metabolites.acac_c: -1.0,     #Acetoacetate
    #WLmodelActn.metabolites.h_c: -1.0, 
    WLmodelActn.metabolites.act_c: 1.0,
    WLmodelActn.metabolites.co2_c: 1.0
})
print(adc.check_mass_balance())    #ensure correct mass balance
print(adc.reaction)                #Ensure correct reaction

{}
acac_c --> act_c + co2_c


In [9]:
actT = Reaction("ACTt")
actT.name = "Acetone transporter"
actT.add_metabolites({ 
    WLmodelActn.metabolites.act_c: -1.0,
    WLmodelActn.metabolites.act_e: 1.0
})
actT

Reaction identifier,ACTt
Name,Acetone transporter
Memory address,0x07fd160d662b0
Stoichiometry,act_c --> act_e Acetone --> Acetone
GPR,
Lower bound,0.0
Upper bound,1000.0


In [10]:
act_ex = Reaction("EX_act_e")
act_ex.name = "Acetone exchange"
act_ex.add_metabolites({ 
    WLmodelActn.metabolites.act_e: -1.0,
})
act_ex

Reaction identifier,EX_act_e
Name,Acetone exchange
Memory address,0x07fd160d66198
Stoichiometry,act_e --> Acetone -->
GPR,
Lower bound,0.0
Upper bound,1000.0


In [11]:
WLmodelActn.add_reactions([ctfAB_ac, adc, actT, act_ex])

## Add alternative acetone production pathway, Lanzatech

In [12]:
WLmodelActn.add_metabolites(Metabolite(id='aactp_c',formula = 'C4H5O6P', name = 'Acetoacetyl phosphate', compartment = 'c' ))

In [13]:
ptacac = Reaction("ptacac")
ptacac.name = "Phosphate acetoacetyltransferase"

ptacac.add_metabolites({
    WLmodelActn.metabolites.aacoa_c: -1.0,   #Acetoacetyl_CoA
    WLmodelActn.metabolites.pi_c: -1.0,      #Phosphate
    WLmodelActn.metabolites.aactp_c: 1.0,    #Acetoacetyl_P
    WLmodelActn.metabolites.coa_c: 1.0       #Coenzyme A
})
print(ptacac.check_mass_balance())    #ensure correct mass balance
print(ptacac.bounds)                  #Ensure correct bounds
print(ptacac.reaction)                #Ensure correct reaction

{}
(0.0, 1000.0)
aacoa_c + pi_c --> aactp_c + coa_c


In [14]:
acack = Reaction("acack")
acack.name = "acetoacetate kinase"

acack.add_metabolites({
    WLmodelActn.metabolites.aactp_c: -1.0,   #Acetoacetyl_P
    WLmodelActn.metabolites.adp_c: -1.0,     #ADP C10H12N5O10P2
    WLmodelActn.metabolites.h_c: -1.0,       #proton
    WLmodelActn.metabolites.acac_c: 1.0,     #Acetoacetate
    WLmodelActn.metabolites.atp_c: 1.0       #ATP C10H12N5O13P3
})
print(acack.check_mass_balance())    #ensure correct mass balance
print(acack.bounds)                  #Ensure correct bounds
print(acack.reaction)                #Ensure correct reaction

{}
(0.0, 1000.0)
aactp_c + adp_c + h_c --> acac_c + atp_c


In [15]:
WLmodelActn.add_reactions([ptacac, acack])

## Change medium, run FBA, and visualize in Escher

In [16]:
medium = WLmodel.medium

medium['EX_co2_e'] = 10
medium['EX_co_e'] = 0
medium['EX_ac_e'] = 0
medium['EX_etoh_e'] = 0
medium['EX_btd_RR_e'] = 0
medium['EX_but_e'] = 0
medium['EX_btoh_e'] = 0
medium['EX_h2_e'] = 1000

WLmodel.medium = medium

In [17]:
WLmodelActn.reactions.CODH_ACS.bounds = 1,1
WLmodelActn.objective = "EX_act_e"
solution = WLmodelActn.optimize()
WLmodelActn.optimize()

,fluxes,reduced_costs
H2td,0.0,-0.0
H2Ot,-2.0,-0.0
CO2t,1.5,0.0
COt,0.0,0.0
ETOHt,0.0,-0.0
...,...,...
ADC,0.5,0.0
ACTt,0.5,0.0
EX_act_e,0.5,0.0
ptacac,0.5,0.0


In [18]:
Builder(model = WLmodelActn, map_json = '../../Models/EscherMaps/WL_core_Actn_SK.json', reaction_data = solution.fluxes.to_dict())


Builder(reaction_data={'H2td': 0.0, 'H2Ot': -2.0, 'CO2t': 1.5, 'COt': 0.0, 'ETOHt': 0.0, 'BTDt_RR': 0.0, 'BUTt…

In [19]:
cobra.io.write_sbml_model(WLmodelActn, "../../Models/WLPathway_sink.xml")